> About this Dataset:

You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

toxic
severe_toxic
obscene
threat
insult
identity_hate

In [2]:
import pandas as pd

from pyspark import SparkConf, SparkContext, HiveContext
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression

In [3]:
# Build a spark context
conf = SparkConf().setAppName("NLP Practise")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

In [6]:
def to_spark_df(fin):
    """
    Parse a filepath to a spark dataframe using the pandas api.
    
    Parameters
    ----------
    fin : str
        The path to the file on the local filesystem that contains the csv data.
        
    Returns
    -------
    df : pyspark.sql.dataframe.DataFrame
        A spark DataFrame containing the parsed csv data.
    """
    df = pd.read_csv(fin)
    df.fillna("", inplace=True)
    df = hc.createDataFrame(df)
    return(df)

train = to_spark_df("train.csv")
test = to_spark_df("test.csv")

> Let View our data

In [10]:
train.take(5)

[Row(id='0000997932d777bf', comment_text="Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27", toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0),
 Row(id='000103f0d9cfb60f', comment_text="D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)", toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0),
 Row(id='000113f07ec002fd', comment_text="Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.", toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0),
 Row(id='0001b41b1c6bb37e', comme

> Cols other than Id, Comment_Text

In [14]:
out_cols = [i for i in train.columns if i not in ["id", "comment_text"]]

print(out_cols)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [15]:
train.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0000997932d777bf|Explanation
Why t...|    0|           0|      0|     0|     0|            0|
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|0001b41b1c6bb37e|"
More
I can't ma...|    0|           0|      0|     0|     0|            0|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
only showing top 5 rows



> ### See some toxic comments below

In [23]:
train.filter(F.col('toxic') == 1).show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|
|0005c987bdfc9d4b|Hey... what is it...|    1|           0|      0|     0|     0|            0|
|0007e25b2121310b|Bye! 

Don't look...|    1|           0|      0|     0|     0|            0|
|001810bf8c45bf5f|You are gay or an...|    1|           0|      1|     0|     1|            1|
|00190820581d90ce|FUCK YOUR FILTHY ...|    1|           0|      1|     0|     1|            0|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
only showing top 5 rows



> Tokenization + Counting Words

In [27]:
# Basic sentence tokenizer
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
wordsData = tokenizer.transform(train)

In [28]:
# Count the words in a document
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
tf = hashingTF.transform(wordsData)

> What is output of raw features?

In [30]:
tf.select('rawFeatures').take(2)


[Row(rawFeatures=SparseVector(262144, {19208: 1.0, 23032: 1.0, 24417: 1.0, 25000: 1.0, 29945: 1.0, 32241: 1.0, 32976: 1.0, 37852: 1.0, 46075: 1.0, 59853: 1.0, 72125: 1.0, 77971: 1.0, 81631: 1.0, 82999: 1.0, 83922: 1.0, 91677: 1.0, 97171: 1.0, 100258: 1.0, 101169: 1.0, 103838: 3.0, 110427: 1.0, 113031: 1.0, 113418: 1.0, 135568: 1.0, 139533: 1.0, 140784: 1.0, 145284: 1.0, 151536: 1.0, 164148: 1.0, 169364: 1.0, 176964: 1.0, 182267: 1.0, 192137: 1.0, 193131: 1.0, 229137: 1.0, 230921: 1.0, 231630: 1.0, 244466: 1.0, 246621: 1.0, 249835: 1.0, 253170: 1.0})),
 Row(rawFeatures=SparseVector(262144, {17429: 1.0, 38728: 1.0, 83815: 1.0, 88337: 1.0, 101527: 1.0, 101833: 1.0, 108541: 1.0, 125765: 1.0, 141219: 1.0, 151980: 1.0, 169364: 1.0, 169800: 1.0, 203235: 1.0, 208090: 1.0, 219140: 1.0, 242101: 1.0, 248135: 1.0, 249180: 1.0}))]

> Build the idf model and transform the original token frequencies into their tf-idf counterparts

In [31]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(tf) 
tfidf = idfModel.transform(tf)

In [32]:
tfidf.select("features").first()

Row(features=SparseVector(262144, {19208: 2.244, 23032: 5.0123, 24417: 0.7386, 25000: 5.6813, 29945: 3.0517, 32241: 8.3967, 32976: 5.0285, 37852: 1.7539, 46075: 6.9564, 59853: 3.1525, 72125: 2.2744, 77971: 7.6108, 81631: 3.4198, 82999: 7.5735, 83922: 6.4588, 91677: 0.6965, 97171: 2.0163, 100258: 1.1947, 101169: 1.734, 103838: 1.2127, 110427: 2.1173, 113031: 8.9845, 113418: 2.2023, 135568: 3.5864, 139533: 2.5136, 140784: 3.0483, 145284: 7.6628, 151536: 2.2412, 164148: 6.0064, 169364: 2.4772, 176964: 1.7656, 182267: 8.613, 192137: 3.1018, 193131: 5.6703, 229137: 4.5705, 230921: 2.0429, 231630: 8.2914, 244466: 3.351, 246621: 10.0343, 249835: 6.827, 253170: 2.7021}))

> ### Now we will Build a logistic regression model for the binary toxic column. Use the features column (the tfidf values) as the input vectors, X, and the toxic column as output vector, y.

In [ ]:
REG = 0.1
lr = LogisticRegression(featuresCol="features", labelCol='toxic', regParam=REG)

In [ ]:
lrModel = lr.fit(tfidf)
res_train = lrModel.transform(tfidf)

In [ ]:
res_train.select("id", "toxic", "probability", "prediction").show(20)

In [ ]:
res_train.select(F.col('probability'))